<a href="https://colab.research.google.com/github/zaedulislam/Deep_Learning_A-Z_Hands-On_Artificial_Neural_Networks/blob/main/Section%207%20-%20Building%20a%20CNN/Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network (CNN)

Notion Page: https://www.notion.so/zaedulislam/Deep-Learning-A-Z-Hands-On-Artificial-Neural-Networks-b0f4742181e74b8da987a0db61e0ced9#ea8f4602d93044f786949b38095a407c

### Importing the libraries

In [2]:
# TensorFlow is already preinstalled as a library in Google Colab, but we still need to import it
import tensorflow as tf
# ImageDataGenerator class generate batches of tensor image data with real-time data augmentation
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.7.0'

## Part 1 - Data Processing

### Preprocessing the Training set

In [4]:
# IMAGE AUGMENTATION
# "zoom_range" -> consists of zooming in or zooming out on the images
# "horizontal_flip" -> consists of flipping the images horizontally
# "shear_range" -> Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)

# FEATURE SCALING
# "rescale" -> This will apply feature scaling to each and every single one of your pixels by dividing their value by 255. Each pixel takes a value between zero and 255. 
# So by dividing all of them by 255, we indeed get all the pixel values between zero and one, which is just like a normalization. 
# Feature scaling is absolutely compulsory when training neural networks.

# train_datagen is an instance of ImageDataGenerator class
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


# "target_size" -> Final size of the images, that will be fed into the convolutional neural network. target_size=(150, 150) takes more time to train
# "batch_size" -> The size of the batches, meaning how many images we want to have in each batch. And the 32 is a classic default value.

training_set = train_datagen .flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

### Preprocessing the Test set

In [ ]:
# We're keeping the test data intact like the original ones by not applying any transformation. 
# However, we've to rescale their pixels to avoid information leakage from the test set. The future predict method of CNN will have
# to be applied to the same scaling as the one that was applied on the training set.
test_datagen = ImageDataGenerator(rescale=1./255) 

# "flow_from_directory(...)" access the dataset from the directory
# "target_size", "batch_size", "class_mode" should be same as for the "training_set"

test_test = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')